In [1]:


import pandas as pd
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


c:\Users\migue\Documents\GitHub\Practica_BAIN_4\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [3]:
# Modelo más liviano (entra en Colab): FLAN-T5 base
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Función de predicción
def predict_flan(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
    output = model.generate(**inputs, max_new_tokens=50)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded.strip()

# Inicializar listas
predictions_base = []
predictions_zero_shot = []
predictions_few_shot = []
predictions_cot = []

# Loop de predicción
for row in df.itertuples():
    opinion = row.opinion
    label = row.label

    prompt_base = f"Clasifica la siguiente opinión en positiva, negativa o neutral: {opinion}"
    
    zero_shot_prompt = f"Clasifica la opinión siguiente como positiva, negativa o neutral:\n\"{opinion}\""
    
    few_shot_prompt = f"""
        Clasifica el sentimiento de la siguiente opinión como positiva, negativa o neutral.
        Ejemplos:
        Opinión: "Me encanta este producto. Es increíble." → Sentimiento: positivo
        Opinión: "Este servicio es terrible, nunca lo recomendaría." → Sentimiento: negativo
        Opinión: "El artículo está bien, pero no es excepcional." → Sentimiento: neutral
        Opinión: "{opinion}" → Sentimiento:"""

    cot_prompt = f"""
        Analiza el siguiente texto y decide si el sentimiento expresado es positivo, negativo o neutral.
        Texto: "{opinion}"
        Razonamiento paso a paso:
        """

    # Predecir cada tipo
    res_zero = predict_flan(zero_shot_prompt)
    res_few = predict_flan(few_shot_prompt)
    res_base = predict_flan(prompt_base)
    res_cot = predict_flan(cot_prompt)

    def assign_sentiment(text):
        normalized_response = text.lower().strip()

        palabras_positivas = ["positivo", "positiva", "positive"]
        palabras_negativas = ["negativo", "negativa", "negative"]
        palabras_neutras = ["neutral", "neutro", "neutra"]

        if any(word in normalized_response for word in palabras_positivas):
            return "positiva"
        elif any(word in normalized_response for word in palabras_negativas):
            return "negativa"
        elif any(word in normalized_response for word in palabras_neutras):
            return "neutral"
        else:
            return "desconocida"

    predictions_zero_shot.append(assign_sentiment(res_zero))
    predictions_few_shot.append(assign_sentiment(res_few))
    predictions_base.append(assign_sentiment(res_base))
    predictions_cot.append(assign_sentiment(res_cot))

    print(f"Opinión: {opinion} - Label: {label}")
    print(f" → Zero-shot: {predictions_zero_shot[-1]}")
    print(f" → Few-shot: {predictions_few_shot[-1]}")
    print(f" → Chain-of-Thought: {predictions_cot[-1]}")
    print(f" → Base: {predictions_base[-1]}\n")

# Agregar al DataFrame
df['prompt_base'] = df['opinion'].apply(lambda x: f"Clasifica la siguiente opinión en positiva, negativa o neutral: {x}")
df['prediccion_zero_shot'] = predictions_zero_shot
df['prediccion_few_shot'] = predictions_few_shot
df['prediccion_cot'] = predictions_cot

# Mostrar resultados
print(df[['opinion', 'label', 'prompt_base', 'prediccion_zero_shot', 'prediccion_few_shot', 'prediccion_cot']])


Opinión: Es un gran servicio - Label: positiva
 → Zero-shot: positiva
 → Few-shot: positiva
 → Chain-of-Thought: positiva
 → Base: positiva

Opinión: Es regular - Label: neutral
 → Zero-shot: positiva
 → Few-shot: neutral
 → Chain-of-Thought: desconocida
 → Base: desconocida

Opinión: Es un producto útil - Label: positiva
 → Zero-shot: positiva
 → Few-shot: positiva
 → Chain-of-Thought: positiva
 → Base: desconocida

Opinión: Es un fraude - Label: negativa
 → Zero-shot: positiva
 → Few-shot: negativa
 → Chain-of-Thought: negativa
 → Base: positiva

Opinión: Me gusta mucho - Label: positiva
 → Zero-shot: positiva
 → Few-shot: positiva
 → Chain-of-Thought: positiva
 → Base: positiva

Opinión: Es un desastre - Label: negativa
 → Zero-shot: positiva
 → Few-shot: negativa
 → Chain-of-Thought: negativa
 → Base: positiva

Opinión: Es un mal servicio - Label: negativa
 → Zero-shot: negativa
 → Few-shot: negativa
 → Chain-of-Thought: negativa
 → Base: negativa

Opinión: Es un producto inútil - 

In [4]:
aciertos_df = pd.DataFrame({
    'prompt_base': predictions_base,
    'zero_shot': predictions_zero_shot,
    'few_shot': predictions_few_shot,
    'chain_of_thought': predictions_cot
})

aciertos = {
    'prompt_base': (aciertos_df['prompt_base'] == df['label']).sum(),
    'zero_shot': (aciertos_df['zero_shot'] == df['label']).sum(),
    'few_shot': (aciertos_df['few_shot'] == df['label']).sum(),
    'chain_of_thought': (aciertos_df['chain_of_thought'] == df['label']).sum()
}

print(f"Aciertos con la prompt base: {aciertos['prompt_base']} - Ratio: {aciertos['prompt_base'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt zero shot: {aciertos['zero_shot']} - Ratio: {aciertos['zero_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt few shots: {aciertos['few_shot']}- Ratio: {aciertos['few_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt chain of thought: {aciertos['chain_of_thought']} - Ratio: {aciertos['chain_of_thought'] / len(aciertos_df):.2f}")

Aciertos con la prompt base: 19 - Ratio: 0.63
Aciertos con la prompt zero shot: 21 - Ratio: 0.70
Aciertos con la prompt few shots: 25- Ratio: 0.83
Aciertos con la prompt chain of thought: 24 - Ratio: 0.80
